In [1]:
#Match N Merge Notebook
#This notebook depends on flexx (to install - conda install flexx)
#To run the Match N Merge solution, "Run All"

In [2]:
import os
cwd = os.getcwd()

In [3]:
from flexx.webruntime import launch

In [4]:
#this launches the UI,must launch in firefox
#set default download file path to the current notebook folder
fileToLaunch = "file:///" + os.getcwd() + "/matchnmerge.html"
launch(fileToLaunch, 'firefox')

INFO flexx.webruntime: launching BrowserRuntime


In [5]:
#clean up
#clear all files in variable removeFiles
#create status.txt
import os
removeFiles = "filesReady.txt, status.txt, columnMapping.csv, results.csv, results.txt"
filesToRemove = removeFiles.split(",")
for file in filesToRemove:
    try:
        os.remove(file)
        print("removed " + file)
    except OSError:
        print("attemped processing of file " + file)


attemped processing of file filesReady.txt
attemped processing of file  status.txt
attemped processing of file  columnMapping.csv
attemped processing of file  results.csv
attemped processing of file  results.txt


In [6]:
#create status.txt
with open('status.txt','w') as f:
    f.write("")

In [7]:
#function to prepend line to file
def line_prepender(filename, line):
    import time
    tm = time.strftime('%H:%M:%S')
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + tm + " : " + content)

In [8]:
#function to write to status.txt
#write to status whatever you want
def writeToStatus(line):
    line_prepender("status.txt", line)

In [9]:
import time
tm = time.strftime('%a, %d %b %Y %H:%M:%S')
line_prepender("status.txt", tm + ":Starting execution ...")
    

In [10]:
# Wait for a file called, "filesReady.txt".  This is the file that is generated when the user has selected the two datasets
# This file will contain two entries, separated by a comma.  They correspond to the dataset files (data set 1 and 2).
# The files will be located at the same location as this notebook
# This code waits for the existence of this file before proceeding
import os.path
import time

file_path = "filesReady.txt"
while not os.path.exists(file_path):
    time.sleep(5);
    if os.path.isfile(file_path):
        continue;
    else:
        time.sleep(1);



ERROR flexx.webruntime: runtime process stopped (1), stdout:



In [11]:
def columnJoiner():
    
    writeToStatus("importing packages")
    
    # no warnings
    import warnings
    warnings.filterwarnings('ignore')

    # the usuals
    import numpy as np
    import pandas as pd

    df = pd.read_csv('filesReady.txt', header=None)
    
    # visualization
    import matplotlib.pyplot as plt

    # machine learning
    from sklearn import preprocessing
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC, LinearSVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB # Gaussian Naive Bays
    from sklearn.linear_model import Perceptron
    from sklearn.linear_model import SGDClassifier #stochastic gradient descent
    from sklearn.tree import DecisionTreeClassifier

    import xgboost as xgb

    # evaluation
    from sklearn.metrics import confusion_matrix

    writeToStatus("importing datasets & transforming dataframes")

    # read dataset 1 - we will train with this dataset
    file1 = str(df.iloc[0,0])
    df1a = pd.read_csv(file1, encoding='latin-1')
    df1a.head()

    # flatten values into one column & create df with column name = key1
    df1b = pd.DataFrame({'key1': df1a.values.flatten()})
    df1b.head()

    # list of column names from file1
    list1 = list(df1a)

    # matrix of column names cycled through the data
    matrix1 = np.transpose(np.tile(np.eye(len(list1), dtype= int), len(df1b.index)//len(list1)))
    dfmatrix1 = pd.DataFrame(matrix1, columns=[list1])

    # matrix merged with flattened file
    df1c = pd.merge(df1b, dfmatrix1, left_index=True, right_index=True, how='right')
    df1c.head()

    # drop NaN in key1 column, reset index
    df1d = df1c.dropna(subset = ['key1']).reset_index(drop=True)
    df1d.head()

    # read dataset 2 - we will validate with this dataset
    file2 = str(df.iloc[0,1])
    df2a = pd.read_csv(file2, encoding='latin-1')
    df2a.tail()

    # flatten values into one column & create df with column name = key2
    df2b = pd.DataFrame({'key2': df2a.values.flatten()})
    df2b.head()

    # list of column names from file2
    list2 = list(df2a)

    # matrix of column names cycled through the data
    matrix2 = np.transpose(np.tile(np.eye(len(list2), dtype= int), len(df2b.index)//len(list2)))
    dfmatrix2 = pd.DataFrame(matrix2, columns=[list2])

    # matrix merged with flattened data
    df2c = pd.merge(df2b, dfmatrix2, left_index=True, right_index=True, how='right')

    # drop NaN in key2 column, reset index
    df2d = df2c.dropna(subset = ['key2']).reset_index(drop=True)
    df2d.head()

    # next part is feature engineering

    writeToStatus("commence feature engineering")

    from collections import Counter
    import string

    def count_letters(word, valid_letters=string.ascii_letters):
        count = Counter(word)
        return sum(count[letter] for letter in valid_letters)

    def count_digits(word, valid_letters=string.digits):
        count = Counter(word)
        return sum(count[digits] for digits in valid_letters)

    def count_whitespace(word, valid_letters=string.whitespace):
        count = Counter(word)
        return sum(count[whitespace] for whitespace in valid_letters)

    def count_punctuation(word, valid_letters=string.punctuation):
        count = Counter(word)
        return sum(count[punctuation] for punctuation in valid_letters)

    df1d['key1'] = df1d['key1'].astype('str')
    df1d['len'] = df1d['key1'].apply(lambda x: len(x))
    df1d['let'] = df1d['key1'].apply(count_letters)
    df1d['num'] = df1d['key1'].apply(count_digits)
    df1d['ws'] = df1d['key1'].apply(count_whitespace)
    df1d['punc'] = df1d['key1'].apply(count_punctuation)
    df1d['%let'] = df1d.let/df1d.len
    df1d['%num'] = df1d.num/df1d.len
    df1d['%ws'] = df1d.ws/df1d.len
    df1d['%punc'] = df1d.punc/df1d.len
    
    df2d['key2'] = df2d['key2'].astype('str')
    df2d['len'] = df2d['key2'].apply(lambda x: len(x))
    df2d['let'] = df2d['key2'].apply(count_letters)
    df2d['num'] = df2d['key2'].apply(count_digits)
    df2d['ws'] = df2d['key2'].apply(count_whitespace)
    df2d['punc'] = df2d['key2'].apply(count_punctuation)
    df2d['%let'] = df2d.let/df2d.len
    df2d['%num'] = df2d.num/df2d.len
    df2d['%ws'] = df2d.ws/df2d.len
    df2d['%punc'] = df2d.punc/df2d.len

    # for now, drop the last nine columns (len:%punc) in dataset 1
    df1matrix = df1d.iloc[:,:-9].drop('key1',axis=1)

    # loop df1matrix and fill in column names for 1s
    for y in df1matrix:
        df1matrix[y] = [y if ele  == 1 
        else 0 for ele in df1matrix[y]]

    writeToStatus("create X's & Y's")
        
    # create Y1
    Y1 = pd.DataFrame({'Y1': df1matrix.replace(0, np.nan).bfill(1).iloc[:, 0]})
    Y1 = Y1.reset_index(drop=True)
    Y1.head()

    # create X1
    X1 = df1d.iloc[:,-9:]
    X1.head()

    # for now, drop the last nine columns (len:%punc) in dataset 2
    df2matrix = df2d.iloc[:,:-9].drop('key2',axis=1)

    # loop df2matrix and fill in column names for 1s
    for y in df2matrix:
        df2matrix[y] = [y if ele  == 1 
        else 0 for ele in df2matrix[y]]

    # create Y2
    Y2 = pd.DataFrame({'Y2': df2matrix.replace(0, np.nan).bfill(1).iloc[:, 0]})
    Y2 = Y2.reset_index(drop=True)
    Y2.head()

    # create X2
    X2 = df2d.iloc[:,-9:]
    X2.head()

    # next is the training & validation

    writeToStatus("creating model based on dataset1")

    # create model, use SVC as its traiing accuracy is fairly high
    model = SVC(probability=True)
    model.fit(X1, Y1)

    # make predictions on X2 based on X1/Y1 model
    Y2_pred = pd.DataFrame(data = model.predict(X2))
    Y2_pred.head()

    writeToStatus("making predictions on dataset2")

    # show max accuracies per row per column
    dfmax = pd.Series(pd.DataFrame(model.predict_proba(X2)).max(axis=1))
    dfmax.head()

    # place Y2 predictions and Y2 actuals side by side
    Y2['Y2_pred'] = Y2_pred
    df = Y2.rename(index=str, columns={"Y2": "Y2_actual", "Y2_pred": "Y2_pred"})
    df.head()

    # show the count of each pair of Y2 prediction and Y2 actual
    df = pd.crosstab(df.Y2_actual,df.Y2_pred).replace(0,np.nan).\
         stack().reset_index().rename(columns={0:'Count'})

    # shows the total count of each Y2 actual category
    df = df.join(df.groupby('Y2_actual')["Count"].sum(), on = 'Y2_actual', rsuffix = 'All')

    # calculate relative accruacy based on total predicted per category/total count per category
    dfAcc = df[['Count']].div(df.CountAll, axis=0)
    df['Relative_Match_Accuracy'] = dfAcc

    # drop Count and CountAll columns
    df = df.loc[df.reset_index().groupby(['Y2_actual'])['Relative_Match_Accuracy'].idxmax()].drop(['Count', 'CountAll'], axis=1)

    # sort column order
    df = df[['Y2_actual', 'Y2_pred', 'Relative_Match_Accuracy']]
    
    # rename columns
    df.columns = ['Base_Column', 'Matched_Column', 'Relative_Match_Accuracy']

    writeToStatus("Match N Merge For Columns Completed")
    
    # write to columnMapping.csv
    df.to_csv('columnMapping.csv', index=False)

In [12]:
columnJoiner()

C:\Users\rlim\AppData\Local\Continuum\Anaconda3\envs\data-x\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
#TODO:   wait for a file called "columnMapping_user.csv".
# This the file that is generated by the UI when the user reviews and acknowledges column mappings

import os.path
import time

file_path = "columnMapping_user.csv"
while not os.path.exists(file_path):
    time.sleep(5);
    if os.path.isfile(file_path):
        continue;
    else:
        time.sleep(1);

In [14]:
#this is Rob's code
def rowJoiner2():
    
    import numpy as np
    import pandas as pd
    import difflib as dl
    import math
    from functools import partial
    import os
    import string
    
    def str_diff(tarString, testString) :
        seq = dl.SequenceMatcher(None,tarString, str(testString))
        ratioNum = seq.ratio()
        return ratioNum
    
    def most_likely(testDF, tarString):
        tempDF = pd.DataFrame()
        tempDF['distance'] = np.NAN
        tempDF['distance'] = testDF.apply(partial(str_diff,tarString))
        maxValue = tempDF['distance'].max()
        maxIndex = tempDF['distance'].idxmax()
        #print('maxValue', maxValue)
        int(maxIndex)
        #print('maxIndex', maxIndex)
        return maxIndex
    
    def most_likelyValue(testDF, tarString):
        tempDF = pd.DataFrame()
        tempDF['distance'] = np.NAN
        tempDF['distance'] = testDF.apply(partial(str_diff,tarString))
        maxValue = tempDF['distance'].max()
        maxIndex = tempDF['distance'].idxmax()
        #print('maxValue', maxValue)
        int(maxIndex)
        #print('maxIndex', maxIndex)
        return maxValue
    
    writeToStatus("Starting row merge")
    
    df = pd.read_csv('filesReady.txt', header=None)
    
    file1 = str(df.iloc[0,0])
    firstDF = pd.read_csv(file1, encoding='latin-1', index_col=None)
    
    file2 = str(df.iloc[0,1])
    secondDF = pd.read_csv(file2, encoding='latin-1', index_col=None)
    
   
    topXRows = 1000
        
    mappedColumns = pd.read_csv("columnMapping_user.csv", encoding='latin-1')
    
    #print('TOTAL DF\n',mappedColumns)
    
    mappedColumns.loc[mappedColumns.iloc[:,-2]=='Do Not Match','Matched_Column'] = np.NAN
    
    mappedColumns.loc[(mappedColumns.iloc[:,-2]!='Do Not Match') & (mappedColumns.iloc[:,-2]!='Keep Match'),'Matched_Column'] = mappedColumns.iloc[:,-2]
    
    mappingDF = pd.DataFrame()
    
    
    ########## If error on reading, uncomment the following row and... 
    #mappingDF['df1Name'] = mappedColumns.index
    
    ########## comment the following row
    mappingDF['df2Name'] = mappedColumns['Base_Column']
    
    mappingDF['df1Name'] = mappedColumns['Matched_Column']
    
    mappingDF.dropna(inplace=True)
    
    #print(mappingDF)

    origFirstCol = list(firstDF.columns)
    orgSecCol= list(secondDF.columns)
    
    matchColClean = []
    
    for cols in mappingDF['df1Name']:
        if((firstDF[cols].dtype == 'float64') | (firstDF[cols].dtype == 'int')) :
            matchColClean.append(cols)
        else:
            newColName = cols + 'Stripped'
            matchColClean.append(newColName)
            firstDF[newColName] = firstDF[cols].str.replace('[^\w\s]','')
            firstDF[newColName] = firstDF[newColName].str.lower()
    
    mappingDF['df1NamClean'] = pd.Series(matchColClean, index=mappingDF.index)
    matchColClean = []
    
    for cols in mappingDF['df2Name']:
        if((secondDF[cols].dtype == 'float64') | (secondDF[cols].dtype == 'int')) :
            matchColClean.append(cols)
        else:
            newColName = cols + 'Stripped'
            matchColClean.append(newColName)
            secondDF[newColName] = secondDF[cols].str.replace('[^\w\s]','')
            secondDF[newColName] = secondDF[newColName].str.lower()
    
    mappingDF['df2NamClean'] = pd.Series(matchColClean, index=mappingDF.index)

    firstDF['bigKey'] = ""
    secondDF['bigKey'] = ""
    for index, cols in mappingDF.iterrows():
        #print(cols)
        firstDF['bigKey'] = firstDF['bigKey'].astype(str) + firstDF[cols['df1NamClean']].astype(str)
        secondDF['bigKey'] = secondDF['bigKey'].astype(str) + secondDF[cols['df2NamClean']].astype(str)
    
    #print(firstDF['bigKey'].head())
    
    output = firstDF.copy()
    
    writeToStatus("Aligning rows")
    
    output['matchIndex'] = firstDF.loc[:topXRows,'bigKey'].apply(partial(most_likely, secondDF['bigKey']))
    
    results = pd.DataFrame()
    
    results = output.join(secondDF,on='matchIndex',how='right',rsuffix='_ds2',lsuffix='_ds1')
    
    results['matchCertainty'] = results.loc[:topXRows,'bigKey_ds1'].apply(partial(most_likelyValue, results['bigKey_ds2']))
    
    dropCol = ['bigKey_ds1','bigKey_ds2']
    
    for cols in list(results.columns):
        if 'Stripped' in cols :
            dropCol.append(cols)
    
    keepCol = list(set(results.columns) - set(list(dropCol)))
    
    results = results[keepCol]
    
    writeToStatus("Creating output file")
    
    for cols in results:
        if cols in origFirstCol:
            newColName = cols + '_ds1'
            results = results.rename(columns={cols:newColName})
        if cols in orgSecCol:
            newColName = cols + '_ds2'
            results = results.rename(columns={cols:newColName})
    
    results.to_csv('results.csv', encoding='latin-1')
    writeToStatus("Match N Merge For Rows Completed")
    
    return True

In [15]:
#this is Rob's code
def rowJoiner():
    import numpy as np
    import pandas as pd
    import difflib as dl
    import math
    from functools import partial
    import os
    import string

    def str_diff(tarString, testString) :
        #returns the ratio difference between 2 strings
        seq = dl.SequenceMatcher(None,tarString, str(testString))
        ratioNum = seq.ratio()
        return ratioNum

    def most_likely(testDF, tarString):
        #identifies which string in a DF is most like the target string.  Returns the index.
        tempDF = pd.DataFrame()
        tempDF['distance'] = np.NAN
        tempDF['distance'] = testDF.apply(partial(str_diff,tarString))
        maxValue = tempDF['distance'].max()
        maxIndex = tempDF['distance'].idxmax()
            #print('maxValue', maxValue)
        int(maxIndex)
            #print('maxIndex', maxIndex)
        return maxIndex

    def most_likelyValue(testDF, tarString):
        #identifies which string in a DF is most like the target string.  Returns the ratio value.
        tempDF = pd.DataFrame()
        tempDF['distance'] = np.NAN
        tempDF['distance'] = testDF.apply(partial(str_diff,tarString))
        maxValue = tempDF['distance'].max()
        maxIndex = tempDF['distance'].idxmax()
            #print('maxValue', maxValue)
        int(maxIndex)
            #print('maxIndex', maxIndex)
        return maxValue

    writeToStatus("Starting row merge")

    #Reading in the two files and which columns are supposed to align.  Incorporates User corrections.
    df = pd.read_csv('filesReady.txt', header=None)

    file1 = str(df.iloc[0,0])
    firstDF = pd.read_csv(file1, encoding='latin-1', index_col=None)

    file2 = str(df.iloc[0,1])
    secondDF = pd.read_csv(file2, encoding='latin-1', index_col=None)


    topXRows = 15

    mappedColumns = pd.read_csv("columnMapping_user.csv", encoding='latin-1')

        #print('TOTAL DF\n',mappedColumns)

    mappedColumns.loc[mappedColumns.iloc[:,-2]=='Do Not Match','Matched_Column'] = np.NAN

    mappedColumns.loc[(mappedColumns.iloc[:,-2]!='Do Not Match') & (mappedColumns.iloc[:,-2]!='Keep Match'),'Matched_Column'] = mappedColumns.iloc[:,-2]

    mappingDF = pd.DataFrame()

     ########## If error on reading, uncomment the following row and... 
    #mappingDF['df1Name'] = mappedColumns.index

        ########## comment the following row
    mappingDF['df2Name'] = mappedColumns['Base_Column']
    mappingDF['df1Name'] = mappedColumns['Matched_Column']

    mappingDF.dropna(inplace=True)

        #print(mappingDF)

    origFirstCol = list(firstDF.columns)
    orgSecCol= list(secondDF.columns)

    matchColClean = []

    #Provides basic string cleaning by converting to lowercase and removing punctuation
    for cols in mappingDF['df2Name']:
        if((firstDF[cols].dtype == 'float64') | (firstDF[cols].dtype == 'int')) :
            matchColClean.append(cols)
        else:
            newColName = cols + 'Stripped'
            matchColClean.append(newColName)
            firstDF[newColName] = firstDF[cols].str.replace('[^\w\s]','')
            firstDF[newColName] = firstDF[newColName].str.lower()

    mappingDF['df1NamClean'] = pd.Series(matchColClean, index=mappingDF.index)
    matchColClean = []

    for cols in mappingDF.index.values:
        if((secondDF[cols].dtype == 'float64') | (secondDF[cols].dtype == 'int')) :
            matchColClean.append(cols)
        else:
            newColName = cols + 'Stripped'
            matchColClean.append(newColName)
            secondDF[newColName] = secondDF[cols].str.replace('[^\w\s]','')
            secondDF[newColName] = secondDF[newColName].str.lower()

    mappingDF['df2NamClean'] = pd.Series(matchColClean, index=mappingDF.index)

    #Combines all overlapped columns into a single string that is compared against the same combined string
    #in the second data set
    firstDF['bigKey'] = ""
    secondDF['bigKey'] = ""
    for index, cols in mappingDF.iterrows():
        #print(cols)
        firstDF['bigKey'] = firstDF['bigKey'].astype(str) + firstDF[cols['df1NamClean']].astype(str)
        secondDF['bigKey'] = secondDF['bigKey'].astype(str) + secondDF[cols['df2NamClean']].astype(str)

    output = firstDF.copy()

    writeToStatus("Aligning rows")

    output['matchIndex'] = firstDF.loc[:topXRows,'bigKey'].apply(partial(most_likely, secondDF['bigKey']))

    #Creates a new DF with the first and second DFs merged based on alignment
    results = pd.DataFrame()

    results = output.join(secondDF,on='matchIndex',how='right',rsuffix='_ds2',lsuffix='_ds1')

    results['matchCertainty'] = results.loc[:topXRows,'bigKey_ds1'].apply(partial(most_likelyValue, results['bigKey_ds2']))

    #Cleans up Results by dropping temporary columns
    dropCol = ['bigKey_ds1','bigKey_ds2']

    for cols in list(results.columns):
        if 'Stripped' in cols :
            dropCol.append(cols)

    keepCol = list(set(results.columns) - set(list(dropCol)))

    results = results[keepCol]

    writeToStatus("Creating results output file")

    #Creating output files
    for cols in results:
        if cols in origFirstCol:
            newColName = cols + '_ds1'
            results = results.rename(columns={cols:newColName})
        if cols in orgSecCol:
            newColName = cols + '_ds2'
            results = results.rename(columns={cols:newColName})

    results.to_csv('results.csv', encoding='latin-1')
    writeToStatus("Match N Merge For Rows Completed")


In [16]:
rowJoiner()

In [17]:
#rowJoiner()